In [1]:
#Import files module
import pandas as pd
import csv
import re                     ##Regular expressions
import glob
import os
import logging
import numpy as np
import datetime
#from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

C:\Users\Lenovo\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\Lenovo\Anaconda2\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implement

In [2]:
#logging mechanism for debugging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
#Mapping months to numeric values
months = {'01':'Jan','02':'Feb','03':'Mar','04':'Apr','05':'May','06':'June','07':'July','08':'Aug','09':'Sept','10':'Oct','11':'Nov','12':'Dec'}

In [13]:
import pandas as pd

'''path = r'D:\Dharmit\Final Year Project\Clean data Vikhroli\0071\Up' 
allfiles = glob.glob(path + "/*.csv")
Holiday_colnames = ['Holiday', 'Date', 'Day', 'Month']
Holiday_data = pd.read_csv('D:\Dharmit\Final Year Project\List of holidays.csv', names=Holiday_colnames)
feature_names =  ['Time Slot','Trips','Passengers','Day','isHoliday']'''

#Importing files from specific folders and list of holidays
bus_no='3860'
path = r'D:/Dharmit/Final Year Project/Clean data Vikhroli/'+bus_no+'/Up' 
down_path = r'D:/Dharmit/Final Year Project/Clean data Vikhroli/'+bus_no+'/Down' 
allfiles = glob.glob(path + "/*.csv")
down_allfiles=glob.glob(down_path + "/*.csv")
Holiday_colnames = ['Holiday', 'Date', 'Day', 'Month']
Holiday_data = pd.read_csv('D:\Dharmit\Final Year Project\List of holidays.csv', names=Holiday_colnames)
Holiday_2017 = pd.read_csv('D:\Dharmit\Final Year Project\List of holidays 2017.csv', names=Holiday_colnames)
feature_names =  ['Time Slot','Trips','Passengers','Day','isHoliday']

In [14]:
#Making separate dataframes for each separate timeslot over many days
df1=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df2=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df3=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df4=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df5=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df6=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df7=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df8=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df9=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df10=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df11=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df12=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df13=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df14=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df15=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df16=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df17=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df18=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df19=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df20=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20]

#Dicing only requird data from list of holidays
holiday_day = Holiday_data['Day']
holiday_month = Holiday_data['Month']
holiday_day_2017 = Holiday_2017['Day']
holiday_month_2017 = Holiday_2017['Month']

In [15]:
Timeslot_values= ['00:00:01-05:00:00','05:00:01-06:00:00','06:00:01-07:00:00','07:00:01-08:00:00','08:00:01-09:00:00','09:00:01-10:00:00','10:00:01-11:00:00','11:00:01-12:00:00','12:00:01-13:00:00','13:00:01-14:00:00','14:00:01-15:00:00','15:00:01-16:00:00','16:00:01-17:00:00','17:00:01-18:00:00','18:00:01-19:00:00','19:00:01-20:00:00','20:00:01-21:00:00','21:00:01-22:00:00','22:00:01-23:00:00','23:00:01-24:00:00']
Days = {'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7}

#Obtaining the day and date details for current day 
now = datetime.datetime.now()   
current_month = months[str(now.month).zfill(2)] 
current_day = now.day
isCurrDay_holiday = 0
current_dow = Days[now.strftime("%A")]           #Obtaining current day of the week
for i in range(1,23):                            #checking if there is holiday today
    if(holiday_day[i]==current_day and current_month==holiday_month[i]):
        isCurrDay_holiday=1

for files in allfiles:
    #Obtaining Day and date details from filename
    base = os.path.basename(files)
    day = os.path.splitext(base)[0][29:]
    date = os.path.splitext(base)[0][26:28]

    month = os.path.splitext(base)[0][23:25]
    year = os.path.splitext(base)[0][18:22]
    #Determining whether holiday or not
    holiday = 0
    if year=='2017':
        for i in range(1,22):
            if(str(holiday_day_2017[i]).zfill(2)==date and months[month]==holiday_month_2017[i]):
            #print'Holiday 2017'
                holiday=1
    else:
        for i in range(1,23):
            if(holiday_day[i]==date and months[month]==holiday_month[i]):
                holiday=1
            
    #Pre-processing data into required format
    day = Days[day]                                        #Mapping day to numeric value
    with open(files,'rb') as f:
        timeslot = []
        trips = []
        passengers = []
        reader = csv.reader(f, delimiter=",")
        rows = [r for r in reader]
        for x in rows[1:]:
            timeslot.append(x[1])
            trips.append(x[2])
            if(x[3]==''):
                x[3]=0
            passengers.append(x[3])
 
    #Adding data into the dataframes created previously according to time slots
    for j in xrange(len(timeslot)):
        df[j]= df[j].append(pd.Series([timeslot[j],trips[j],int(passengers[j]),day,holiday], index = ['Time Slot','Trips','Passengers','Day','isHoliday']), ignore_index=True)
        df[j]=df[j].replace('',0)                      #Replacing NAN values by 0

In [16]:
print df[6]                                       #To check whether data has been successfully pre-processed

             Time Slot Trips  Passengers  Day  isHoliday
0    10:00:01-11:00:00     4          19    7          1
1    10:00:01-11:00:00     2          29    1          1
2    10:00:01-11:00:00     5         164    2          0
3    10:00:01-11:00:00     4         155    3          0
4    10:00:01-11:00:00     6         399    4          0
5    10:00:01-11:00:00     2         112    5          0
6    10:00:01-11:00:00     2         121    6          0
7    10:00:01-11:00:00     2          50    7          0
8    10:00:01-11:00:00     3         202    1          0
9    10:00:01-11:00:00     6         302    2          0
10   10:00:01-11:00:00     4         178    3          0
11   10:00:01-11:00:00     3         182    4          0
12   10:00:01-11:00:00     3         138    5          0
13   10:00:01-11:00:00     3          88    6          0
14   10:00:01-11:00:00     2          39    7          0
15   10:00:01-11:00:00     4         184    1          0
16   10:00:01-11:00:00     6   

In [9]:
for i in xrange(20):
    for j in xrange(len(df[i]['Time Slot'])):
        df[i]['Time Slot'][j]=int(df[i]['Time Slot'][j][:2])

C:\Users\Lenovo\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [37]:
print df[6]

   Time Slot Trips  Passengers  Day  isHoliday
0         10     0         150    4          0
1         10     2         127    1          0
2         10     1         207    5          0
3         10     0         132    2          0
4         10     2         155    6          0
5         10     2         124    3          0
6         10     1          93    7          0
7         10     0         171    4          0
8         10     0         181    1          0
9         10     0         135    5          0
10        10     1          98    2          0
11        10     1         162    6          0
12        10     1         155    3          0
13        10     0           9    7          0
14        10     1         152    4          0
15        10     0          90    1          0
16        10     2         238    5          0
17        10     2         212    2          0
18        10     1         195    6          0
19        10     1         173    3          0
20        10 

In [38]:
train_all_features = df[10][feature_names][:32].drop(['Trips','Time Slot'], axis=1).values
test_all_features=df[10][feature_names][32:].drop(['Trips','Time Slot'], axis=1).values
all_classes = df[10]['Trips'].values
print train_all_features,all_classes

[[ 35   4   0]
 [148   1   0]
 [173   5   0]
 [  0   2   0]
 [ 49   6   0]
 [  0   3   0]
 [144   7   0]
 [ 82   4   0]
 [129   1   0]
 [102   5   0]
 [123   2   0]
 [138   6   0]
 [ 73   3   0]
 [ 69   7   0]
 [ 63   4   0]
 [ 50   1   0]
 [ 92   5   0]
 [118   2   0]
 [ 79   6   0]
 [159   3   0]
 [ 65   7   0]
 [162   4   0]
 [ 97   1   0]
 [117   2   1]
 [ 79   3   0]
 [138   1   0]
 [102   2   0]
 [111   5   0]
 [183   6   0]
 [ 56   7   0]
 [ 89   1   0]
 [112   2   0]] ['1' '1' '0' '0' '1' 0 '2' '1' '0' '1' '2' '1' '0' '1' '1' '2' '1' '0' '1'
 '3' '2' '0' '1' '1' '1' '1' '0' '1' '1' '1' '1' '1' '1' '1' '1' '4' '3'
 '1' '1' '0']


In [39]:
train_labels = keras.utils.to_categorical(df[10]['Trips'][:32], 5)
test_labels = keras.utils.to_categorical(df[10]['Trips'][32:], 5)
print df[10]['Trips'][:32].argmax()



19


In [40]:
from theano import ifelse
model = Sequential()
model.add(Dense(4, activation='relu', input_shape=(3,)))
model.add(Dense(2, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))

In [41]:
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [42]:
history = model.fit(train_all_features, train_labels,
                    batch_size=30,
                    epochs=10,
                    verbose=2,
                    validation_data=(test_all_features, test_labels))

Train on 32 samples, validate on 8 samples
Epoch 1/10
 - 0s - loss: 12.2127 - acc: 0.0625 - val_loss: 12.2021 - val_acc: 0.1250
Epoch 2/10
 - 0s - loss: 12.3610 - acc: 0.0313 - val_loss: 12.0180 - val_acc: 0.1250
Epoch 3/10
 - 0s - loss: 12.6467 - acc: 0.0313 - val_loss: 11.9533 - val_acc: 0.1250
Epoch 4/10
 - 0s - loss: 11.5019 - acc: 0.0625 - val_loss: 11.8406 - val_acc: 0.1250
Epoch 5/10
 - 0s - loss: 11.9062 - acc: 0.0625 - val_loss: 11.7995 - val_acc: 0.1250
Epoch 6/10
 - 0s - loss: 12.5976 - acc: 0.0313 - val_loss: 11.7619 - val_acc: 0.1250
Epoch 7/10
 - 0s - loss: 12.7086 - acc: 0.0313 - val_loss: 11.6709 - val_acc: 0.1250
Epoch 8/10
 - 0s - loss: 11.9272 - acc: 0.0313 - val_loss: 11.6244 - val_acc: 0.1250
Epoch 9/10
 - 0s - loss: 11.7910 - acc: 0.0625 - val_loss: 11.5861 - val_acc: 0.1250
Epoch 10/10
 - 0s - loss: 10.8415 - acc: 0.0625 - val_loss: 11.4666 - val_acc: 0.1250


In [43]:
score = model.evaluate(test_all_features, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

('Test loss:', 11.466622352600098)
('Test accuracy:', 0.125)


In [44]:
print test_labels
print 'train'
print train_x.dtype




[[ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]]
train


NameError: name 'train_x' is not defined

In [45]:
for x in range(0,8):
    test =test_all_features[x].reshape(1,3)
    print model.predict(test)
    predicted_cat = model.predict(test).argmax()
    label = test_labels[x].argmax()
    if (predicted_cat != label):
        print('Prediction: %d Label: %d' % (predicted_cat, label))

[[  1.29223310e-09   2.33744863e-08   3.14844357e-07   9.99622583e-01
    3.77006130e-04]]
Prediction: 3 Label: 1
[[  5.25249419e-12   2.05648068e-10   5.57119284e-09   9.99954820e-01
    4.51463420e-05]]
Prediction: 3 Label: 1
[[  2.71341036e-04   8.74377321e-04   2.47213454e-03   9.55069125e-01
    4.13130820e-02]]
Prediction: 3 Label: 1
[[  3.91552457e-15   4.20918617e-13   2.84421549e-11   9.99997139e-01
    2.81081043e-06]]
Prediction: 3 Label: 4
[[  2.74474559e-07   2.34076288e-06   1.59666433e-05   9.97010708e-01
    2.97072181e-03]]
[[  2.21982086e-18   6.80846354e-16   1.18806508e-13   9.99999881e-01
    1.57467554e-07]]
Prediction: 3 Label: 1
[[  2.71705836e-09   4.42817694e-08   5.42760972e-07   9.99497294e-01
    5.02053939e-04]]
Prediction: 3 Label: 1
[[  5.91076154e-04   1.70340284e-03   4.35304921e-03   9.38184202e-01
    5.51682301e-02]]
Prediction: 3 Label: 0


In [32]:
current_dow=2
current_month = 'Feb'
current_day =13
isCurrDay_holiday=1

In [17]:
#Linear Regression module to predict passengers and number of trips
import sklearn
import numpy as np
#import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
scale = StandardScaler()    
from theano import ifelse
total_loss=0
total_acc=0

err_sum=0
trips=[]
up_pass=[]

# To obtain average error measure across all dataframes
for j in xrange(len(timeslot)):
    print 'Timeslot',j+1,':'
    pass_data=np.asarray(df[j][['Day','isHoliday']])
    pass_labels=np.asarray(df[j]['Passengers'])
    target=[[current_dow,isCurrDay_holiday]]                      # Example target list to check output format and debugging

    passenger_predictor = LinearRegression(normalize=True)          #Linear regression model to predict passenger frequency
    passenger_predictor.fit(pass_data, pass_labels)
    #err_measure = passenger_predictor.score(pass_data, pass_labels, sample_weight=None)  #Error in passenger frequency prediction
    pass_pred=passenger_predictor.predict(target)
    up_pass.append(round(pass_pred))
    target=np.append(pass_pred,target)
    print target                                                    # Target features for predicting number of trips
    limit=int(round(0.8*len(df[j])))
    train_all_features = df[j][feature_names][:limit].drop(['Trips','Time Slot'], axis=1).values #create training data with trips and time slot dropped
    test_all_features=df[j][feature_names][limit:].drop(['Trips','Time Slot'], axis=1).values    #create testing data with trips and time slot dropped
    all_classes = df[j]['Trips'].values
    #print train_all_features,all_classes
    train_labels = keras.utils.to_categorical(df[j]['Trips'][:limit],8) #create categorical data df[i]['Trips'][:32].argmax()
    test_labels = keras.utils.to_categorical(df[j]['Trips'][limit:],8)  #create categorical data
    #model creation 
    model = Sequential()
    #input layer
    model.add(Dense(4, activation='relu', input_shape=(3,)))
    #hidden layer
    model.add(Dense(2, activation='relu')) 
    #drop 20% nodes
    model.add(Dropout(0.2))
    #output layer
    model.add(Dense(8, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
    
    history = model.fit(train_all_features, train_labels,batch_size=30,epochs=10,verbose=2,
                        validation_data=(test_all_features, test_labels))
    
    score = model.evaluate(test_all_features, test_labels, verbose=0)
    total_loss+=score[0]
    total_acc+=score[1]
    predicted_trips=0

    target=target.reshape(1,3)
    predicted_trips += model.predict(target).argmax()
    trips.append(predicted_trips)
    print('Predicted trips for timeslot: %d is: %d' % (j+1,predicted_trips))
    
for i in range(len(trips)):
        if trips[i]=='':
            trips[i]=0   
trips = [0 if x is None else x for x in trips]
    

print float(total_loss)/20
print float(total_acc)/20
    

Timeslot 1 :
[ 6.31296195  4.          0.        ]


IndexError: index 8 is out of bounds for axis 1 with size 8

In [22]:
#Making separate dataframes for each separate timeslot over many days
df1=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df2=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df3=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df4=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df5=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df6=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df7=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df8=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df9=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df10=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df11=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df12=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df13=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df14=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df15=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df16=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df17=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df18=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df19=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df20=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20]

In [23]:
Timeslot_values= ['00:00:01-05:00:00','05:00:01-06:00:00','06:00:01-07:00:00','07:00:01-08:00:00','08:00:01-09:00:00','09:00:01-10:00:00','10:00:01-11:00:00','11:00:01-12:00:00','12:00:01-13:00:00','13:00:01-14:00:00','14:00:01-15:00:00','15:00:01-16:00:00','16:00:01-17:00:00','17:00:01-18:00:00','18:00:01-19:00:00','19:00:01-20:00:00','20:00:01-21:00:00','21:00:01-22:00:00','22:00:01-23:00:00','23:00:01-24:00:00']
Days = {'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7}

#Obtaining the day and date details for current day 
now = datetime.datetime.now()   
current_month = months[str(now.month).zfill(2)] 
current_day = now.day
isCurrDay_holiday = 0
current_dow = Days[now.strftime("%A")]           #Obtaining current day of the week
for i in range(1,24):                            #checking if there is holiday today
    if(holiday_day[i]==current_day and current_month==holiday_month[i]):
        isCurrDay_holiday=1

for files in down_allfiles:
    #Obtaining Day and date details from filename
    base = os.path.basename(files)
    base = os.path.basename(files)
    day = os.path.splitext(base)[0][31:]
    date = os.path.splitext(base)[0][29:31]
    month = os.path.splitext(base)[0][26:28]
    
    #Determining whether holiday or not
    year = os.path.splitext(base)[0][21:25]
    #Determining whether holiday or not
    holiday = 0
    if year=='2017':
        for i in range(1,22):
            if(str(holiday_day_2017[i]).zfill(2)==date and months[month]==holiday_month_2017[i]):
            #print'Holiday 2017'
                holiday=1
    else:
        for i in range(1,23):
            if(holiday_day[i]==date and months[month]==holiday_month[i]):
                holiday=1
            
    #Pre-processing data into required format
    day = Days[day]                                        #Mapping day to numeric value
    with open(files,'rb') as f:
        timeslot = []
        trips = []
        passengers = []
        reader = csv.reader(f, delimiter=",")
        rows = [r for r in reader]
        for x in rows[1:]:
            timeslot.append(x[1])
            trips.append(x[2])
            if(x[3]==''):
                x[3]=0
            passengers.append(x[3])
 
    #Adding data into the dataframes created previously according to time slots
    for j in xrange(len(timeslot)):
        df[j]= df[j].append(pd.Series([timeslot[j],trips[j],int(passengers[j]),day,holiday], index = ['Time Slot','Trips','Passengers','Day','isHoliday']), ignore_index=True)
        df[j]=df[j].replace('',0)                      #Replacing NAN values by 0

In [24]:
#Linear Regression module to predict passengers and number of trips
import sklearn
import numpy as np
#import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
scale = StandardScaler()    
from theano import ifelse
total_loss=0
total_acc=0

err_sum=0
down_trips=[]
down_pass=[]

# To obtain average error measure across all dataframes
for j in xrange(len(timeslot)):
    print 'Timeslot',j+1,':'
    pass_data=np.asarray(df[j][['Day','isHoliday']])
    pass_labels=np.asarray(df[j]['Passengers'])
    target=[[current_dow,isCurrDay_holiday]]                      # Example target list to check output format and debugging

    passenger_predictor = LinearRegression(normalize=True)          #Linear regression model to predict passenger frequency
    passenger_predictor.fit(pass_data, pass_labels)
    #err_measure = passenger_predictor.score(pass_data, pass_labels, sample_weight=None)  #Error in passenger frequency prediction
    pass_pred=passenger_predictor.predict(target)
    down_pass.append(round(pass_pred))
    target=np.append(pass_pred,target)
    print target                                                    # Target features for predicting number of trips
    train_all_features = df[j][feature_names][:36].drop(['Trips','Time Slot'], axis=1).values #create training data with trips and time slot dropped
    test_all_features=df[j][feature_names][36:].drop(['Trips','Time Slot'], axis=1).values    #create testing data with trips and time slot dropped
    all_classes = df[j]['Trips'].values
    #print train_all_features,all_classes
    train_labels = keras.utils.to_categorical(df[j]['Trips'][:36],8) #create categorical data df[i]['Trips'][:32].argmax()
    test_labels = keras.utils.to_categorical(df[j]['Trips'][36:],8)  #create categorical data
    #model creation 
    model = Sequential()
    #input layer
    model.add(Dense(4, activation='relu', input_shape=(3,)))
    #hidden layer
    model.add(Dense(2, activation='relu')) 
    #drop 20% nodes
    model.add(Dropout(0.2))
    #output layer
    model.add(Dense(8, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
    
    history = model.fit(train_all_features, train_labels,batch_size=30,epochs=10,verbose=2,
                        validation_data=(test_all_features, test_labels))
    
    score = model.evaluate(test_all_features, test_labels, verbose=0)
    total_loss+=score[0]
    total_acc+=score[1]
    predicted_trips=0

    target=target.reshape(1,3)
    predicted_trips += model.predict(target).argmax()
    down_trips.append(predicted_trips)
    print('Predicted trips for timeslot: %d is: %d' % (j+1,predicted_trips))
    
for i in range(len(down_trips)):
        if down_trips[i]=='':
            down_trips[i]=0   
down_trips = [0 if x is None else x for x in down_trips]
    

print float(total_loss)/20
print float(total_acc)/20
    

Timeslot 1 :
[ 13.7237524   3.          0.       ]
Train on 36 samples, validate on 101 samples
Epoch 1/10
 - 0s - loss: 1.9025 - acc: 0.6667 - val_loss: 1.9595 - val_acc: 0.4752
Epoch 2/10
 - 0s - loss: 1.9138 - acc: 0.7500 - val_loss: 1.9571 - val_acc: 0.6733
Epoch 3/10
 - 0s - loss: 1.8602 - acc: 0.7500 - val_loss: 1.9564 - val_acc: 0.9010
Epoch 4/10
 - 0s - loss: 1.8697 - acc: 0.8889 - val_loss: 1.9520 - val_acc: 0.9010
Epoch 5/10
 - 0s - loss: 1.8776 - acc: 0.8611 - val_loss: 1.9456 - val_acc: 0.9010
Epoch 6/10
 - 0s - loss: 1.7914 - acc: 0.9167 - val_loss: 1.9403 - val_acc: 0.9010
Epoch 7/10
 - 0s - loss: 1.8719 - acc: 0.8611 - val_loss: 1.9369 - val_acc: 0.9010
Epoch 8/10
 - 0s - loss: 1.8121 - acc: 0.9167 - val_loss: 1.9303 - val_acc: 0.9010
Epoch 9/10
 - 0s - loss: 1.7712 - acc: 0.8889 - val_loss: 1.9269 - val_acc: 0.9010
Epoch 10/10
 - 0s - loss: 1.7968 - acc: 0.8889 - val_loss: 1.9224 - val_acc: 0.9010
Predicted trips for timeslot: 1 is: 1
Timeslot 2 :
[ 12.60718701   3.    

Epoch 4/10
 - 0s - loss: 2.3649 - acc: 0.1667 - val_loss: 2.7822 - val_acc: 0.3125
Epoch 5/10
 - 0s - loss: 2.3584 - acc: 0.3056 - val_loss: 2.7178 - val_acc: 0.3125
Epoch 6/10
 - 0s - loss: 2.3060 - acc: 0.2778 - val_loss: 2.6645 - val_acc: 0.3125
Epoch 7/10
 - 0s - loss: 2.1341 - acc: 0.3333 - val_loss: 2.6123 - val_acc: 0.3125
Epoch 8/10
 - 0s - loss: 2.1169 - acc: 0.2500 - val_loss: 2.5888 - val_acc: 0.3125
Epoch 9/10
 - 0s - loss: 2.2062 - acc: 0.1944 - val_loss: 2.5524 - val_acc: 0.3125
Epoch 10/10
 - 0s - loss: 2.2098 - acc: 0.1667 - val_loss: 2.5213 - val_acc: 0.3125
Predicted trips for timeslot: 9 is: 2
Timeslot 10 :
[ 131.17861155    3.            0.        ]
Train on 36 samples, validate on 96 samples
Epoch 1/10
 - 0s - loss: 2.0789 - acc: 0.1667 - val_loss: 2.0740 - val_acc: 0.2292
Epoch 2/10
 - 0s - loss: 2.0740 - acc: 0.1111 - val_loss: 2.0712 - val_acc: 0.0208
Epoch 3/10
 - 0s - loss: 2.0709 - acc: 0.1111 - val_loss: 2.0688 - val_acc: 0.0208
Epoch 4/10
 - 0s - loss: 2.06

Epoch 10/10
 - 0s - loss: 2.9854 - acc: 0.6389 - val_loss: 3.0986 - val_acc: 0.6354
Predicted trips for timeslot: 17 is: 0
Timeslot 18 :
[ 13.54897776   3.           0.        ]
Train on 36 samples, validate on 96 samples
Epoch 1/10
 - 0s - loss: 2.0973 - acc: 0.2778 - val_loss: 2.0607 - val_acc: 0.1771
Epoch 2/10
 - 0s - loss: 2.0298 - acc: 0.0833 - val_loss: 2.0150 - val_acc: 0.2813
Epoch 3/10
 - 0s - loss: 2.0220 - acc: 0.3056 - val_loss: 1.9904 - val_acc: 0.3229
Epoch 4/10
 - 0s - loss: 2.0056 - acc: 0.3056 - val_loss: 1.9369 - val_acc: 0.3229
Epoch 5/10
 - 0s - loss: 1.9560 - acc: 0.3611 - val_loss: 1.9223 - val_acc: 0.3438
Epoch 6/10
 - 0s - loss: 1.9815 - acc: 0.3611 - val_loss: 1.9110 - val_acc: 0.3438
Epoch 7/10
 - 0s - loss: 1.8912 - acc: 0.3889 - val_loss: 1.8720 - val_acc: 0.3542
Epoch 8/10
 - 0s - loss: 1.8695 - acc: 0.4444 - val_loss: 1.8426 - val_acc: 0.3542
Epoch 9/10
 - 0s - loss: 1.8855 - acc: 0.5000 - val_loss: 1.8248 - val_acc: 0.3854
Epoch 10/10
 - 0s - loss: 1.921

In [25]:
forecasted_trips = pd.DataFrame(columns=['Time Slot','Categorical_Trips','Categorical_pass','Categorical_Trips_Down','Categorical_pass_Down'])
save_path=r'D:/Dharmit/Final Year Project/Forecasted Trips/'
for j in xrange(len(timeslot)):
    forecasted_trips= forecasted_trips.append(pd.Series([timeslot[j],trips[j],up_pass[j],down_trips[j],down_pass[j]], index = ['Time Slot','Categorical_Trips','Categorical_pass','Categorical_Trips_Down','Categorical_pass_Down']), ignore_index=True)
filename ='Forecasted_trips_'+bus_no+'_'+str(now.month).zfill(2) +'_'+str(now.day)+'_'+str(day)+'_KERAS'
forecasted_trips.to_csv(save_path + filename +'.csv')

In [33]:
print trips
print down_trips


['1', '', '', '2', '2', '2', '1', '1', '1', '2', '2', '1', '0', '0', '0', '0', '0', '0', '', '']
[6, 0, 2, 2, 4, 1, 0, 6, 3, 7, 5, 7, 2, 0, 7, 7, 5, 5, 3, 0]
